# ADMETlab2.0 task autocommit
This code automates the submission of SMILES data to ___ADMETlab2.0___ for predictive analysis, resulting in a significant enhancement to the usability and scalability of the ___CrossFuse-XGBoost___ model.

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## 1. Get CSRF token

In [12]:
index_file = "ADMETlab_output/admet2_index.html"
url = "https://admetmesh.scbdd.com/service/evaluation/index"

In [8]:
# Send RPC (remote procedure call) command to ADMETlab2.0 and put down its response in a file.
index_response = requests.get(url)
with open("admet2_index.html", 'w+', encoding='utf-8') as wt:
    wt.writelines(index_response.text)

In [9]:
# Parse tokens returned by ADMETLab2.0 server.
with open(index_file, 'r+', encoding='utf-8') as f:
    content = f.read()
soup = BeautifulSoup(content, 'html.parser')
csrf_token_input = soup.find_all('input', {"name": "csrfmiddlewaretoken"})
csrf_token = csrf_token_input[0]['value']

## 2. Submit task to _ADMETlab2.0_

In [9]:
in_smiles = "CC(C)OC(=O)CC(=O)CSC1=C(C=C2CCCC2=N1)C#N"

In [10]:
# Get ADMET features for given compounds from ADMETlab2.0 server via RPC.
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
    , 'Accept-Encoding': 'gzip, deflate, br'
    , 'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6'
    , 'Content-Type': 'application/x-www-form-urlencoded'
    , 'Cookie': 'csrftoken=' + csrf_token
    , 'Host': 'admetmesh.scbdd.com'
    , 'Origin': 'https://admetmesh.scbdd.com'
    , 'Referer': 'https://admetmesh.scbdd.com/service/evaluation/index'
    , 'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"'
    , 'sec-ch-ua-mobile': '?1'
    , 'sec-ch-ua-platform': "Windows"
    , 'Sec-Fetch-Dest': 'document'
    , 'Sec-Fetch-Mode': 'navigate'
    , 'Sec-Fetch-Site': 'same-origin'
    , 'Sec-Fetch-User': '?1'
    , 'Upgrade-Insecure-Requests': '1'
    ,
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
params = {'csrfmiddlewaretoken': csrf_token, 'smiles': in_smiles, 'method': '1'}
response = requests.post("https://admetmesh.scbdd.com/service/evaluation/cal", headers=headers, data=params)
admet2_res_file = 'ADMETlab_output/admet2_res_file.html'
with open(admet2_res_file, 'w+', encoding='utf-8') as wt:
    wt.writelines(response.text)

In [12]:
with open(admet2_res_file, 'r+', encoding='utf-8') as f:
    res_html = f.read()
res_soup = BeautifulSoup(res_html, 'html.parser')

In [13]:
for i in res_soup.find_all('a', attrs={"style": "text-decoration: none;"}):
    i_href = i['href']
    if i_href.startswith('/static/files/'):
        print(i_href)
        break
download_href = "https://admetmesh.scbdd.com" + i_href
down_response = requests.get(download_href)

In [13]:
admet2_csv_file = "ADMETlab_output/admet2_res_file.csv"
with open(admet2_csv_file, 'w+', encoding='utf-8') as wt:
    wt.writelines(down_response.text)

/static/files/evaluation/result/tmp/tmpy53iwts41695719845.csv


## 3. Display obtained ADMET features

In [15]:
in_admet2_df = pd.read_csv(admet2_csv_file)
in_admet2_df.head()

,smiles,LogS,LogD,LogP,Pgp-inh,Pgp-sub,HIA,F(20%),F(30%),Caco-2,...,MCE-18,Natural Product-likeness,Alarm_NMR,BMS,Chelating,PAINS,Lipinski,Pfizer,GSK,GoldenTriangle
0,CC(C)OC(=O)CC(=O)CSC1=C(C=C2CCCC2=N1)C#N,-3.958,2.247,2.385,0.815,0.003,0.005,0.004,0.008,-4.694,...,29.25,-1.447,2,0,0,0,Accepted,Accepted,Accepted,Accepted
